- latest `trl` version
    - https://huggingface.co/docs/trl/main/en/grpo_trainer

In [12]:
from IPython.display import Image

In [2]:
from datasets import load_dataset
dataset = load_dataset("trl-lib/tldr", split="train")

In [5]:
dataset

Dataset({
    features: ['prompt', 'completion'],
    num_rows: 116722
})

## GRPOConfig & GRPOTrainer

In [9]:
import os
os.environ["NCCL_P2P_DISABLE"] = "1"
os.environ["NCCL_IB_DISABLE"] = "1"

In [11]:
from rich.pretty import pprint
pprint(GRPOConfig(output_dir='test'))

GRPOConfig(
│   output_dir='test',
│   overwrite_output_dir=False,
│   do_train=False,
│   do_eval=False,
│   do_predict=False,
│   eval_strategy=<IntervalStrategy.NO: 'no'>,
│   prediction_loss_only=False,
│   per_device_train_batch_size=1,
│   per_device_eval_batch_size=8,
│   per_gpu_train_batch_size=None,
│   per_gpu_eval_batch_size=None,
│   gradient_accumulation_steps=8,
│   eval_accumulation_steps=None,
│   eval_delay=0,
│   torch_empty_cache_steps=None,
│   learning_rate=1e-06,
│   weight_decay=0.0,
│   adam_beta1=0.9,
│   adam_beta2=0.999,
│   adam_epsilon=1e-08,
│   max_grad_norm=1.0,
│   num_train_epochs=3.0,
│   max_steps=-1,
│   lr_scheduler_type=<SchedulerType.LINEAR: 'linear'>,
│   lr_scheduler_kwargs={},
│   warmup_ratio=0.0,
│   warmup_steps=0,
│   log_level='passive',
│   log_level_replica='warning',
│   log_on_each_node=True,
│   logging_dir='test/runs/Feb06_22-44-00_whaow',
│   logging_strategy=<IntervalStrategy.STEPS: 'steps'>,
│   logging_first_step=False,
│   logging_steps=500,
│   logging_nan_inf_filter=True,
│   save_strategy=<SaveStrategy.STEPS: 'steps'>,
│   save_steps=500,
│   save_total_limit=None,
│   save_safetensors=True,
│   save_on_each_node=False,
│   save_only_model=False,
│   restore_callback_states_from_checkpoint=False,
│   no_cuda=False,
│   use_cpu=False,
│   use_mps_device=False,
│   seed=42,
│   data_seed=None,
│   jit_mode_eval=False,
│   use_ipex=False,
│   bf16=False,
│   fp16=False,
│   fp16_opt_level='O1',
│   half_precision_backend='auto',
│   bf16_full_eval=False,
│   fp16_full_eval=False,
│   tf32=None,
│   local_rank=0,
│   ddp_backend=None,
│   tpu_num_cores=None,
│   tpu_metrics_debug=False,
│   debug=[],
│   dataloader_drop_last=False,
│   eval_steps=None,
│   dataloader_num_workers=0,
│   dataloader_prefetch_factor=None,
│   past_index=-1,
│   run_name='test',
│   disable_tqdm=False,
│   remove_unused_columns=False,
│   label_names=None,
│   load_best_model_at_end=False,
│   metric_for_best_model=None,
│   greater_is_better=None,
│   ignore_data_skip=False,
│   fsdp=[],
│   fsdp_min_num_params=0,
│   fsdp_config={'min_num_params': 0, 'xla': False, 'xla_fsdp_v2': False, 'xla_fsdp_grad_ckpt': False},
│   fsdp_transformer_layer_cls_to_wrap=None,
│   accelerator_config=AcceleratorConfig(
│   │   split_batches=False,
│   │   dispatch_batches=None,
│   │   even_batches=True,
│   │   use_seedable_sampler=True,
│   │   non_blocking=False,
│   │   gradient_accumulation_kwargs=None,
│   │   use_configured_state=False
│   ),
│   deepspeed=None,
│   label_smoothing_factor=0.0,
│   optim=<OptimizerNames.ADAMW_TORCH: 'adamw_torch'>,
│   optim_args=None,
│   adafactor=False,
│   group_by_length=False,
│   length_column_name='length',
│   report_to=['tensorboard', 'wandb'],
│   ddp_find_unused_parameters=None,
│   ddp_bucket_cap_mb=None,
│   ddp_broadcast_buffers=None,
│   dataloader_pin_memory=True,
│   dataloader_persistent_workers=False,
│   skip_memory_metrics=True,
│   use_legacy_prediction_loop=False,
│   push_to_hub=False,
│   resume_from_checkpoint=None,
│   hub_model_id=None,
│   hub_strategy=<HubStrategy.EVERY_SAVE: 'every_save'>,
│   hub_token=None,
│   hub_private_repo=None,
│   hub_always_push=False,
│   gradient_checkpointing=False,
│   gradient_checkpointing_kwargs=None,
│   include_inputs_for_metrics=False,
│   include_for_metrics=[],
│   eval_do_concat_batches=True,
│   fp16_backend='auto',
│   evaluation_strategy=None,
│   push_to_hub_model_id=None,
│   push_to_hub_organization=None,
│   push_to_hub_token=None,
│   mp_parameters='',
│   auto_find_batch_size=False,
│   full_determinism=False,
│   torchdynamo=None,
│   ray_scope='last',
│   ddp_timeout=1800,
│   torch_compile=False,
│   torch_compile_backend=None,
│   torch_compile_mode=None,
│   dispatch_batches=None,
│   split_batches=None,
│   include_tokens_per_second=False,
│   include_num_input_tokens_seen=False,
│   neftune_noise_alpha=None,
│   optim_target_modules=None,
│   batch_eval_metrics=False,
│   eval_on_s

- GRPOConfig
    - num_generations=8,
    - per_device_train_batch_size=1, * gradient_accumulation_steps=8,
        - per_device_train_batch_size * gradient_accumulation_steps * world_size ==> train_batch

In [6]:
from trl import GRPOConfig, GRPOTrainer

[2025-02-06 22:42:06,848] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)
 [WARNING]  Please specify the CUTLASS repo directory as environment variable $CUTLASS_PATH
 [WARNING]  sparse_attn requires a torch version >= 1.5 and < 2.0 but detected 2.5
 [WARNING]  using untested triton version (3.1.0), only 1.0.0 is known to be compatible


/home/whaow/anaconda3/lib/python3.10/site-packages/deepspeed/runtime/zero/linear.py:49: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  def forward(ctx, input, weight, bias=None):
/home/whaow/anaconda3/lib/python3.10/site-packages/deepspeed/runtime/zero/linear.py:67: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  def backward(ctx, grad_output):


## GRPO

In [14]:
# bs = 2
# G = 4
Image(url='https://huggingface.co/datasets/trl-lib/documentation-images/resolve/main/grpo_visual.png', width=400)

- GRPO is an **online learning**(on-policy) algorithm, meaning it improves iteratively by using the data generated by the trained model itself during training.
- four main steps:
    - Generating completions,
        - At each training step, we sample a batch of prompts and generate a set of $G$ completions(`num_generations`) for each prompt (denoted as $o_i$).
    - computing the advantage,
        - $\hat A_{i,t}=\frac{r_i-\mu(\mathbf r)}{\sigma(\mathbf r)}$
        - Outcome supervision provides the normalized reward at the end of each output $o_i$ and sets the advantages $\hat A_{i,t}$ of all tokens in the output as the normalized reward
    - **estimating** the KL divergence, (token-level see the figure)
        - https://huggingface.co/docs/trl/main/en/grpo_trainer#estimating-the-kl-divergence
        - `per_token_kl = torch.exp(ref_per_token_logps - per_token_logps) - (ref_per_token_logps - per_token_logps) - 1`
    - and computing the loss.
        - $\pi_{ref}, (\pi_{old}, \pi_\theta)$
        - https://github.com/huggingface/trl/issues/2608
```
# x - x.detach() allows for preserving gradients from x
per_token_loss = torch.exp(per_token_logps - per_token_logps.detach()) * advantages.unsqueeze(1)
per_token_loss = -(per_token_loss - self.beta * per_token_kl)
loss = ((per_token_loss * completion_mask).sum(dim=1) / completion_mask.sum(dim=1)).mean()
```

### reward function

```
def reward_func(completions, **kwargs):
    """Reward function that gives higher scores to longer completions."""
    return [float(len(completion)) for completion in completions]
```
- completions
    - `[bs, G]`
- 20
    - https://github.com/huggingface/trl/issues/2771

### PPO vs. GRPO

在 RLHF 中，我们需要把人类反馈或reward model 对整个序列的打分（例如，一次对话的最终质量分）融合到强化学习训练中。此外，为了让模型在训练时不要偏离参考策略（reference model）太远，我们常常还会引入一个基于 KL 的惩罚项。

因此，在 PPO 里面，最关键的变化在于——如何构造每个 token（每个时间步）的奖励 $r_t$
 。这往往通过下面两步完成：

### training monitor

- You should rely mostly on the reward. And keep an eye on the generations (risk of reward hacking)
    - https://github.com/huggingface/trl/issues/2703